<!-- Autoheader begin -->
<hr/>
<div id="navtitle_1_2_py" style="text-align:center; font-size:16px">I.2 Population Transfer in a Three-Level-System with STIRAP</div>
<hr/>
<table style="width: 100%">
  <tr>
    <th rowspan="2" style="width:33%; text-align:center; font-size:16px">
        <a href="py_exercise_1_1_TLS.ipynb">$\leftarrow$ previous notebook </a><br>
        <a href="py_exercise_1_1_TLS.ipynb" style="font-size:13px">I.1 Population Inversion in a Two-Level-System</a>
    </th>
    <td style="width:33%; text-align:center; font-size:16px">
    </td>
    <th rowspan="2" style="width:33%; text-align:center; font-size:16px">
        <a href="py_exercise_1_3_chirp.ipynb">next notebook $\rightarrow$</a><br>
        <a href="py_exercise_1_3_chirp.ipynb" style="font-size:13px">I.3 Interaction of a Two-Level-System with a Chirped Laser Pulse</a>
    </th>
  </tr>
  <tr style="width: 100%">
    <td style="width:33%; text-align:center; font-size:16px">
        <a href="py_exercise_2_2_lambda.ipynb" style="font-size:13px">II.2 Parameter Optimization for STIRAP</a><br>
        <a href="py_exercise_2_2_lambda.ipynb">$\downarrow$ next part $\downarrow$</a>
    </td>
  </tr>
</table>

<div style="text-align: right;font-size: 16px"><a href="../Julia/jl_exercise_1_2_lambda.ipynb">👉 Julia version</a></div>

---
<!-- Autoheader end -->

# Population Transfer in a Three-Level-System with STIRAP

$\newcommand{tr}[0]{\operatorname{tr}}
\newcommand{diag}[0]{\operatorname{diag}}
\newcommand{abs}[0]{\operatorname{abs}}
\newcommand{pop}[0]{\operatorname{pop}}
\newcommand{aux}[0]{\text{aux}}
\newcommand{opt}[0]{\text{opt}}
\newcommand{tgt}[0]{\text{tgt}}
\newcommand{init}[0]{\text{init}}
\newcommand{lab}[0]{\text{lab}}
\newcommand{rwa}[0]{\text{rwa}}
\newcommand{bra}[1]{\langle#1\vert}
\newcommand{ket}[1]{\vert#1\rangle}
\newcommand{Bra}[1]{\left\langle#1\right\vert}
\newcommand{Ket}[1]{\left\vert#1\right\rangle}
\newcommand{Braket}[2]{\left\langle #1\vphantom{#2} \mid
#2\vphantom{#1}\right\rangle}
\newcommand{op}[1]{\hat{#1}}
\newcommand{Op}[1]{\hat{#1}}
\newcommand{dd}[0]{\,\text{d}}
\newcommand{Liouville}[0]{\mathcal{L}}
\newcommand{DynMap}[0]{\mathcal{E}}
\newcommand{identity}[0]{\mathbf{1}}
\newcommand{Norm}[1]{\lVert#1\rVert}
\newcommand{Abs}[1]{\left\vert#1\right\vert}
\newcommand{avg}[1]{\langle#1\rangle}
\newcommand{Avg}[1]{\left\langle#1\right\rangle}
\newcommand{AbsSq}[1]{\left\vert#1\right\vert^2}
\newcommand{Re}[0]{\operatorname{Re}}
\newcommand{Im}[0]{\operatorname{Im}}$

This notebook introduces the so-called Lambda-system, a three-level system which serves as a slightly more advanced version of the two-level system discussed in [Exercise I.1](py_exercise_1_1_TLS.ipynb). In a Lambda system the levels 1 and 2 and the levels 2 and 3 are coupled, but the levels 1 and 3 are not. Drawing the level schemes with the corresponding couplings then looks a lot like the letter $\Lambda$, which you will see in the course of this notebook! We will consider two pulses interacting with a lambda system, one for each of the two couplings. While a transition from levels 1 to 3 is not directly allowed, it is possible via the intermediate level 2. At the end of this notebook you will learn, that it is even possible to transfer population from level 1 to level 3 without ever populating the intermediate level 2!

## Setup

In [ ]:
import qutip  # QUantum Toolbox In Python
import numpy as np  # package for numerical functions such as cos, sin, etc.
import matplotlib.pylab as plt

# Some utilities for showing hints and solutions
from utils.exercise_1_lambda import *

## The STIRAP Hamiltonian

We consider a three level system with levels labeled 1,2,3. The levels 1-2 and 2-3 are connected via the corresponding transition dipole moments $d_{1,2}$ and $d_{2,3}$. An electric dipole transition between levels 1 and 3 is forbidden, leading to $d_{1,3}=0$.

The system is initialized in the state $\ket{1}$. The goal is to transfer the population to the state $\ket{3}$ by means of coherent control. Furthermore, we choose as an additional goal to avoid populating level 2 as much as possible. This could be physically motivated for example by the state $\ket{2}$ being subject to unwanted decay.

We will use two light fields as controls, targeting the transition 1-2 and 2-3. These are usually called the *Pump* and *Stokes* pulse, respectively.

The Hamiltonian in the lab frame takes the form

\begin{align}\op{H}_{\text{lab}} &= \begin{pmatrix}\varepsilon_{1} & d^{*}_{1,2}E(t)\\ d_{1,2}E(t) & \varepsilon_{2} & d^{*}_{2,3}E(t)\\ & d_{2,3}E(t) & \varepsilon_{3} \end{pmatrix},\end{align}

where $E(t)=E_{\text{P}}(t) + E_{\text{S}}(t)$ is the time-dependent electric control field which consists of both the pump and stokes fields $E_{\text{P/S}}(t)$. The fields have frequencies $\omega_{\text{P}}$ resp. $\omega_{\text{S}}$ and take the form
\begin{align} E_{P/S}(t)&=S_{\text{P/S}}(t)\cos \left(\omega_{\text{P/S}}t\right), \end{align}

with $S_{\text{P/S}}(t)$ the *shape functions*. We go to the rotating frame defined by the following unitary matrix
\begin{align} \op{U} &= e^{i\epsilon_{1}t}\begin{pmatrix}1\\ & e^{i\omega_{P}t}\\ & & e^{i\left(\omega_{P}-\omega_{S}\right)t} \end{pmatrix}. \end{align}

Under this transformation the Hamiltonian transforms as
\begin{align} \op{H}_{\text{rot}}=\op{U}\op{H}_{\text{lab}}\op{U}^{\dagger}+i\dot{\op{U}}\op{U}^{\dagger}. \end{align}

After performing a rotating wave approximation, in which fast oscillating terms like $e^{i(\omega_{\text{P}} \pm \omega_{\text{S}})t}$ are neglected, we obtain the Hamiltonian most commonly found in literature as the starting point of the STIRAP protocol,

\begin{align} \op{H}_{\text{STIRAP}}=\begin{pmatrix}0 & \frac{1}{2}\Omega^{*}_{\text{P}}(t)\\
\frac{1}{2}\Omega_{\text{P}}(t) & \Delta_\text{P} & \frac{1}{2}\Omega^{*}_{\text{S}}(t)\\
& \frac{1}{2}\Omega_{\text{S}}(t) & \Delta_\text{P}-\Delta_\text{S} \end{pmatrix},
\end{align}

where $\Delta_{\text{P}} \equiv (\varepsilon_2 - \varepsilon_1) - \omega_\text{P}$ and $\Delta_S \equiv (\varepsilon_2 - \varepsilon_3) - \omega_\text{S}$ are called *one-photon detunings* and $\delta \equiv \Delta_\text{P} - \Delta_\text{S}$ is the *two-photon detuning*. We also defined $\Omega_{\text{P}}(t) = d_{1,2} S_\text{P}(t)$ resp. $\Omega_{\text{S}}(t) = d_{2,3} S_\text{S}(t)$, the time-dependent *Rabi frequencies*.

The level scheme, pulses and detunings are illustrated in the following figure:

<center><img src="../figures/lambda_system_levels.png" alt="Lambda system considered in this notebook" width="500"></center>

As mentioned in the introduction, this configuration is called Lambda system because the level scheme resembles the letter $\Lambda$!

## The dark state and STIRAP protocol

In the following, we assume the controls to be two-photon resonant, i.e., $\delta = \Delta_\text{P} - \Delta_\text{S} = 0$. We write $\Delta \equiv \Delta_\text{P} = \Delta_\text{S}$.

The three instantaneous eigenstates of $\op{H}_\text{STIRAP}$ can then be expressed as

\begin{align}
\ket{\varphi_{+}} & =\sin\varphi\sin\theta\ket{1}+\cos\varphi\ket{2}+\sin\varphi\cos\theta\ket{3}\\
\ket{\varphi_{-}} & =\cos\varphi\sin\theta\ket{1}-\sin\varphi\ket{2}+\cos\varphi\cos\theta\ket{3}\\
\ket{\varphi_{0}} & =\cos\theta\ket{1}-\sin\theta\ket{3}
\end{align}

with corresponding eigenvalues $\varepsilon_\pm(t) = \frac{\Delta(t)}{2}\pm\frac{1}{2}\sqrt{\Delta^2(t) + \Omega_0^2(t)}$ and $\Omega_0(t)=\sqrt{\Omega_\text{P}^2(t) + \Omega_\text{S}^2(t)}$. The time-dependent angles are given via

\begin{align}
\tan \theta(t) &= \frac{\Omega_\text{P}(t)}{\Omega_\text{S}(t)} \\
\tan 2\varphi(t) &= \frac{\Omega_0 (t)}{\Delta(t)}
\end{align}

Notice that the eigenstate $\ket{\varphi_0}$ does *not* contain any amplitude in state $\ket{2}$. It is therefore immune to any decay affecting $\ket{2}$. Since it is not directly involved in the process, it is called a *dark state*.

Since the dark state is a superposition of $\ket{1}$ and $\ket{3}$ where the respective amplitudes depend on the control amplitudes, we can use the dark state to transfer population from 1 to 3 by having the system adiabatically follow the dark state.

Initially (at $t=-\infty$), we want the system to be in state $\ket{1}$. For $\ket{\phi_0}$, this corresponds to $\theta = 0$ or $\Omega_\text{P} / \Omega_\text{S} \rightarrow 0$.
Eventually (at $t=\infty$), we want the system to be in state $\ket{3}$, demanding $\theta = \pi/2$ or $\Omega_\text{S} / \Omega_\text{P} \rightarrow 0$.

This indicates a counter-intuitive ordering of the two pulses: At first we need to apply the Stokes pulse targeting the 2-3 transition and thereafter we need to apply the Pump pulse on the 1-2 transition.

On top of the correct order of the pulses, we need to make sure that the protocol is slow enough such the population always follows the instantaneous eigenstates. This is known as *adiabaticity*. One can show [1] that the system follows its instantaneous eigenstates adiabatically if

\begin{align}
\Omega_{0}(t)\gg & \left|\dot{\theta}(t)\right|\\
\Leftrightarrow1\gg & \left|\frac{\Omega_{\text{S}}(t)\dot{\Omega}_{\text{P}}(t)-\Omega_{P}(t)\dot{\Omega}_{S}(t)}{\Omega_{0}^{3}(t)}\right|
\end{align}

The above condition can be interpreted as follows: The duration of the pulses can be estimated as $T \approx 1/ |\dot{\theta}|$. The adiabaticity condition can then be written as $\Omega_0 T \gg 1$. Note that $\Omega_0$ is essentially the overall strength of the two pulses. In short: The protocol follows the dark state for very long and very strong pulses.

[1] http://arxiv.org/abs/1605.00224v2

## Implementation using Gaussian pulses

Now let's consider a numerical implementation of the previous theoretical findings where we model the two pulses with Gaussian envelopes. First, we define the pulse shape functions and another function returning the STIRAP Hamiltonian.

We also define the time interval `t_list` for the calculation and its initial state `psi_0`.

In [ ]:
def gaussian_shape(t, kwargs):
    """
    Gaussian shape function centered around kwargs["t_0"] with width kwargs["σ"]^2
    """
    return np.exp(
        -1 / 2 * (t - kwargs["t_0"]) ** 2 / kwargs["σ"] ** 2
    ) / np.sqrt(2 * np.pi * kwargs["σ"])


def pump_shape(t, kwargs):
    """
    Shape function for the pump pulse
    """
    return gaussian_shape(t, {"t_0": kwargs["t_p"], "σ": kwargs["σ_p"]})


def stokes_shape(t, kwargs):
    """
    Shape function for the Stokes pulse
    """
    return gaussian_shape(t, {"t_0": kwargs["t_s"], "σ": kwargs["σ_s"]})


def h_stirap(t, kwargs):
    """
    Function returning the STIRAP Hamiltonian as a QuTiP object
    """
    Ω_p = pump_shape(t, kwargs)
    Ω_s = stokes_shape(t, kwargs)

    out = np.zeros(shape=(3, 3), dtype=complex)
    out[1, 1] = kwargs["Δ"]

    out[0, 1] = kwargs["d_12"] * Ω_p / 2
    out[1, 0] = np.conj(out[0, 1])

    out[1, 2] = kwargs["d_23"] * Ω_s / 2
    out[2, 1] = np.conj(out[1, 2])

    return qutip.Qobj(out)


t_list = np.linspace(-250, 250, 401)  # time-interval

psi_0 = qutip.basis(3, 0)  # initial state
print("Initial state: ")
display(psi_0)

## Problem 1: Choosing parameters for STIRAP

Now we can specify the parameters for the calculation. Below you will find a
set of parameters that does not result in a perfect STIRAP protocol. Play
around with the parameters and try to create the ideal dynamics described above. In fact, the initial parameters we provided for you are already almost correct. There is just one small issue - can you find it?

In [ ]:
kwargs = {
    "t_p": -50,  # Center of the Pump pulse
    "t_s": 50,   # Center of the Stokes pulse
    "σ_p": 50,   # Width of the Pump pulse
    "σ_s": 50,   # Width of the Stokes pulse
    "Δ": 1,      # Single-photon detuning
    "d_12": 25,  # Electric-dipole moment for 1-2 transition / Pump Strength
    "d_23": 25,  # Electric-dipole moment for 2-3 transition / Stokes Strength
}

p1, p2, p3 = (
    qutip.projection(3, 0, 0),
    qutip.projection(3, 1, 1),
    qutip.projection(3, 2, 2),
)
se_result = qutip.sesolve(
    h_stirap, psi_0, t_list, args=kwargs, e_ops=[p1, p2, p3]
)

fig, ax = plt.subplots(1, 2, figsize=(10, 4))

ax[0].plot(t_list, pump_shape(t_list, kwargs), label="pump")
ax[0].plot(t_list, stokes_shape(t_list, kwargs), label="stokes")
ax[0].set_xlabel("time")
ax[0].set_ylabel(r"$\Omega(t)$")
ax[0].legend()

for i_state, population in enumerate(se_result.expect):
    ax[1].plot(t_list, population, label=f"level {i_state+1}")
ax[1].legend()
ax[1].set_xlabel("time")
ax[1].set_ylabel("Populations")
plt.show()

In [ ]:
#problem_1.hint
#problem_1.solution

## Next Steps

After learning about the basics of the STIRAP protocol, you can now try out to find the optimal pulse parameters numerically with optimal control! [Example II.2](py_exercise_2_2_lambda.ipynb) explains how to find the proper parameters to achieve the population inversion discussed above with a gradient-free optimization. [Example III.2](py_exercise_3_2_lambda.ipynb) does the same with a gradient-based approach - Krotov's method.

<!-- Autofooter begin -->

---

[⬆︎ jump to top](#navtitle_1_2_py)
<!-- Autofooter end -->